In [2]:
%pylab inline
#特征选择
#http://www.tuicool.com/articles/ieUvaq

Populating the interactive namespace from numpy and matplotlib


In [4]:
import matplotlib.pyplot as plot

In [14]:
from sklearn.feature_selection import VarianceThreshold
x = [[1,2,3,4],[1,3,4,5],[0,0,0,0],[1,3,4,4]]
p = 0.7#某个值在一维里出现的概率
selector = VarianceThreshold(threshold=p*(1-p))
selector.fit_transform(x)

array([[2, 3, 4],
       [3, 4, 5],
       [0, 0, 0],
       [3, 4, 4]])

In [15]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
iris = load_iris()

In [17]:
print iris

{'target_names': array(['setosa', 'versicolor', 'virginica'], 
      dtype='|S10'), 'data': array([[ 5.1,  3.5,  1.4,  0.2],
       [ 4.9,  3. ,  1.4,  0.2],
       [ 4.7,  3.2,  1.3,  0.2],
       [ 4.6,  3.1,  1.5,  0.2],
       [ 5. ,  3.6,  1.4,  0.2],
       [ 5.4,  3.9,  1.7,  0.4],
       [ 4.6,  3.4,  1.4,  0.3],
       [ 5. ,  3.4,  1.5,  0.2],
       [ 4.4,  2.9,  1.4,  0.2],
       [ 4.9,  3.1,  1.5,  0.1],
       [ 5.4,  3.7,  1.5,  0.2],
       [ 4.8,  3.4,  1.6,  0.2],
       [ 4.8,  3. ,  1.4,  0.1],
       [ 4.3,  3. ,  1.1,  0.1],
       [ 5.8,  4. ,  1.2,  0.2],
       [ 5.7,  4.4,  1.5,  0.4],
       [ 5.4,  3.9,  1.3,  0.4],
       [ 5.1,  3.5,  1.4,  0.3],
       [ 5.7,  3.8,  1.7,  0.3],
       [ 5.1,  3.8,  1.5,  0.3],
       [ 5.4,  3.4,  1.7,  0.2],
       [ 5.1,  3.7,  1.5,  0.4],
       [ 4.6,  3.6,  1. ,  0.2],
       [ 5.1,  3.3,  1.7,  0.5],
       [ 4.8,  3.4,  1.9,  0.2],
       [ 5. ,  3. ,  1.6,  0.2],
       [ 5. ,  3.4,  1.6,  0.4],
       [ 5.2,  3.

In [19]:
x,y = iris.data,iris.target
#chi2 or f_classif分类
#f_regression 用于回归
x = SelectKBest(chi2,k=2).fit_transform(x,y)
print x.shape

(150, 2)


In [27]:
import numpy as np
from scipy.stats import pearsonr
np.random.seed(0)
x= np.random.normal(0,1,200)
x = iris.data
print x.shape
x0,x1,x2,x3= [x[:,i] for i in range(4)]
print pearsonr(x0,x1)

(150, 4)
(-0.10936924995064935, 0.18276521527136649)


In [29]:
#互信息和最大互信息系数
from miniepy import MINE
m = MINE()
x = np.random.uniform(-1,1,100)
m.compute_score(x,x**2)
print m.mic()

ImportError: No module named miniepy

In [10]:
from sklearn.cross_validation import cross_val_score,ShuffleSplit
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestRegressor
import numpy as np
boston = load_boston()
Y = boston["target"]
X = boston["data"]
names = boston["feature_names"]
rf = RandomForestRegressor(n_estimators=20,max_depth=4)
scores = []
#回归问题的特征筛选
for i in range(X.shape[1]):
    score = cross_val_score(rf,X[:,i:i+1],Y,scoring="r2",cv=ShuffleSplit(len(X),3,0.3))
    scores.append((round(np.mean(score),3),names[i]))
print sorted(scores,reverse=True)

[(0.632, 'LSTAT'), (0.59, 'RM'), (0.425, 'NOX'), (0.34, 'PTRATIO'), (0.326, 'TAX'), (0.276, 'INDUS'), (0.26, 'CRIM'), (0.188, 'ZN'), (0.182, 'RAD'), (0.116, 'B'), (0.067, 'DIS'), (0.066, 'AGE'), (0.016, 'CHAS')]


In [21]:
from sklearn.linear_model import LinearRegression,Lasso
from sklearn.preprocessing import StandardScaler
lr = LinearRegression()
scaler = StandardScaler()
X = scaler.fit_transform(X)
lr.fit(X,Y)
print names
def pretty_print_linear(coefs,names = None,sort = False ):
    if names == None:
        names = ["X%s" % x for x in range(len(coefs))]
    lst = zip(coefs,names)
    if sort:
        lst = sorted(lst,key=lambda x:-np.abs(x[0]))
    return "+".join("%s * %s"%(round(coef,3),name) for coef,name in lst)
print pretty_print_linear(lr.coef_,names=names,sort=True)

#与随机森林效果不同的原因是特征与效应变量之间可能不是线性关系，并且特征之间不独立


#加入L1正则
lasso = Lasso(alpha=0.3)
lasso.fit(X,Y)
print pretty_print_linear(lasso.coef_,names=names,sort=True)


#L2正则 可以更好地处理关联性
"""
L2正则化将系数向量的L2范数添加到了损失函数中。由于L2惩罚项中系数是二次方的，这使得L2和L1有着诸多差异，最明显的一点就是，L2正则化会让系数的取值变得平均。对于关联特征，这意味着他们能够获得更相近的对应系数。还是以Y=X1+X2为例，假设X1和X2具有很强的关联，如果用L1正则化，不论学到的模型是Y=X1+X2还是Y=2X1，惩罚都是一样的，都是2 alpha。但是对于L2来说，第一个模型的惩罚项是2 alpha，但第二个模型的是4*alpha。可以看出，系数之和为常数时，各系数相等时惩罚是最小的，所以才有了L2会让各个系数趋于相同的特点。

可以看出，L2正则化对于特征选择来说一种稳定的模型，不像L1正则化那样，系数会因为细微的数据变化而波动。所以L2正则化和L1正则化提供的价值是不同的，L2正则化对于特征理解来说更加有用：表示能力强的特征对应的系数是非零。
"""
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=10)
ridge.fit(X,Y)
print pretty_print_linear(ridge.coef_,names=names,sort=True)

['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']
-3.749 * LSTAT+-3.104 * DIS+2.671 * RM+2.659 * RAD+-2.076 * TAX+-2.062 * PTRATIO+-2.06 * NOX+1.081 * ZN+-0.92 * CRIM+0.857 * B+0.682 * CHAS+0.143 * INDUS+0.021 * AGE
-3.707 * LSTAT+2.992 * RM+-1.757 * PTRATIO+-1.081 * DIS+-0.7 * NOX+0.631 * B+0.54 * CHAS+-0.236 * CRIM+0.081 * ZN+-0.0 * INDUS+-0.0 * AGE+0.0 * RAD+-0.0 * TAX
-3.628 * LSTAT+-2.857 * DIS+2.739 * RM+2.095 * RAD+-1.989 * PTRATIO+-1.815 * NOX+-1.565 * TAX+0.954 * ZN+-0.851 * CRIM+0.851 * B+0.708 * CHAS+-0.039 * INDUS+-0.031 * AGE


/home/bianwj/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:9: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


In [25]:
#随机森林 平均不纯度减少
"""
这里特征得分实际上采用的是 Gini Importance 。使用基于不纯度的方法的时候，
要记住：1、这种方法存在 偏向 ，对具有更多类别的变量会更有利；
2、对于存在关联的多个特征，其中任意一个都可以作为指示器（优秀的特征），并且一旦某个特征被选择之后，
其他特征的重要度就会急剧下降，因为不纯度已经被选中的那个特征降下来了，其他的特征就很难再降低那么多不纯度了，
这样一来，只有先被选中的那个特征重要度很高，其他的关联特征重要度往往较低。在理解数据时，这就会造成误解，
导致错误的认为先被选中的特征是很重要的，而其余的特征是不重要的，
但实际上这些特征对响应变量的作用确实非常接近的（这跟Lasso是很像的）。

需要注意的一点是，关联特征的打分存在不稳定的现象，这不仅仅是随机森林特有的，大多数基于模型的特征选择方法都存在这个问题
"""
rf = RandomForestRegressor()
rf.fit(X,Y)
print sorted(zip(map(lambda x:round(x,4),rf.feature_importances_),names),reverse=True)

#随机特征选择可以消除上面的问题，所以试试咯
rf = RandomForestRegressor(n_estimators=20,max_features=2)
rf.fit(X,Y)
print sorted(zip(map(lambda x:round(x,3),rf.feature_importances_),names))

[(0.4627, 'RM'), (0.318, 'LSTAT'), (0.0827, 'DIS'), (0.0348, 'CRIM'), (0.0303, 'NOX'), (0.0191, 'AGE'), (0.0175, 'PTRATIO'), (0.0138, 'TAX'), (0.0103, 'B'), (0.0046, 'INDUS'), (0.0043, 'RAD'), (0.001, 'ZN'), (0.0009, 'CHAS')]
[(0.007, 'ZN'), (0.011, 'CHAS'), (0.027, 'RAD'), (0.032, 'AGE'), (0.034, 'B'), (0.054, 'TAX'), (0.065, 'DIS'), (0.068, 'CRIM'), (0.076, 'INDUS'), (0.085, 'PTRATIO'), (0.089, 'NOX'), (0.219, 'RM'), (0.232, 'LSTAT')]


In [27]:
#稳定性选择
"""
稳定性选择是一种基于二次抽样和选择算法相结合较新的方法，
选择算法可以是回归、SVM或其他类似的方法。
它的主要思想是在不同的数据子集和特征子集上运行特征选择算法，
不断的重复，最终汇总特征选择结果，
比如可以统计某个特征被认为是重要特征的频率
（被选为重要特征的次数除以它所在的子集被测试的次数）。
理想情况下，重要特征的得分会接近100%。
稍微弱一点的特征得分会是非0的数，而最无用的特征得分将会接近于0。
"""
from sklearn.linear_model import RandomizedLasso
rlasso = RandomizedLasso(alpha=0.025)
rlasso.fit(X, Y)
print sorted(zip(map(lambda x: round(x, 4), rlasso.scores_), 
                 names), reverse=True)

/home/bianwj/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/randomized_l1.py:53: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  for _ in range(n_resampling)):
/home/bianwj/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/randomized_l1.py:53: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  for _ in range(n_resampling)):
/home/bianwj/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/randomized_l1.py:53: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  for _ in range(n_resampling)):
/home/bianwj/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/randomized_l1.py:53: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  for _ in range(n_resampling)):
/home/bianwj/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/randomized_l1.py:53: DeprecationWarning: This function is deprecated. Please

[(1.0, 'RM'), (1.0, 'PTRATIO'), (1.0, 'LSTAT'), (0.61, 'B'), (0.565, 'CHAS'), (0.445, 'CRIM'), (0.34, 'TAX'), (0.25, 'NOX'), (0.2, 'DIS'), (0.145, 'INDUS'), (0.055, 'ZN'), (0.01, 'RAD'), (0.01, 'AGE')]


In [28]:
from sklearn.feature_selec

ImportError: cannot import name SelectFromModel